Backtest on commodities (Brent crude oil). We will use daily data.

More of a conceptual project than anything else, we'll ignore slippage and annoying fees, also assuming perfect liquidity, and so on and so forth.


In [20]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import files
uploaded = files.upload()
uploaded2 = files.upload()

Saving brent-daily.csv to brent-daily (1).csv


Saving wti-daily.csv to wti-daily (1).csv


In [21]:
df = pd.read_csv("brent-daily.csv")
df.head()

,Date,Price
0,1987-05-20,18.63
1,1987-05-21,18.45
2,1987-05-22,18.55
3,1987-05-25,18.60
4,1987-05-26,18.63


This dataset is from 1987-present. I think it would be unwise to use data this far back as regimes change over time. In other words, we probably have too much data and we may encounter problems like the Simpson's paradox (E.g. maybe a strategy is great from 1987-1999 but terrible after that, yet averaged together this effect is hidden).


Of course we still need "enough" data though, so 2 decades worth is probably enough, although pretty arbitrary (but we need to start somewhere) let's just cutoff everything before 2005.

In [22]:
df = df.copy()
df["Date"] = pd.to_datetime(df["Date"])
df = df[df["Date"] >= "2005-01-01"]
df = df.sort_values("Date").drop_duplicates("Date")
df["r"] = df["Price"].pct_change()
df = df.set_index("Date")

In [23]:
# replace NaN with 0
df["r"] = df["r"].fillna(0)

Let's first define our **strategy** with buy and hold as our baseline:

**1.** **First** signal quite simple, will just be binary and 1 if our price is above the *moving average* of say, 50 days. I don't want to COMPLETELY arbitrarily pick a deadband so we will *quantitatively* calculate one by using a cost-based system.

*   Input a list of possible deltas (deadband values)
*   Evaluate performance through a Sharpe ratio on train
*   Pick the delta from the list that maximizes the ratio while slightly penalizing excessive turnover

I feel this system is pretty intuitive (so good for our case) although I'm sure there's a smarter way to go about things. I think 50 days is fair for a commodity (longer typically better?).

\
**2.** **Second** will be mean reversion, i.e. z-score vs rolling mean.

**3.** **Third** Let's define our *position* as long/flat/short = -1/0/1. In simpler terms either we're shorting, completely out, or in

In [24]:
# helpers
def make_trend_positions(df, ma_window=50, delta=0.005):
    """
    Creates trend positions {-1,0,1} from price and moving average with a deadband.
    Returns a Series pos_trend_raw indexed by date.
    """
    ma = df["Price"].rolling(ma_window).mean()
    trend_score = df["Price"] / ma - 1

    pos = np.select(
        [trend_score > delta, trend_score < -delta],
        [1, -1],
        default=0
    )
    return pd.Series(pos, index=df.index, name="pos_trend_raw")


def backtest_from_pos_raw(df, pos_raw, cost_bps=10):
    """
    Takes pos_trend_raw and performs a backtest on it
    """
    out = df[["Price", "r"]].copy()
    out["pos_raw"] = pos_raw

    # Shift 1 day back to avoid lookahead
    out["pos"] = out["pos_raw"].shift(1).fillna(0)

    out["turnover"] = (out["pos"] - out["pos"].shift(1)).abs().fillna(0)
    c = cost_bps / 10000.0
    out["cost"] = c * out["turnover"]

    out["strat_r"] = out["pos"] * out["r"].fillna(0) - out["cost"]
    out["equity"] = (1 + out["strat_r"]).cumprod()
    return out


def sharpe_ann(strat_r, trading_days=252):
    """
    Annual sharp calculation
    """
    strat_r = strat_r.dropna()
    if len(strat_r) < 50:
        return np.nan
    mu = strat_r.mean() * trading_days
    vol = strat_r.std(ddof=0) * np.sqrt(trading_days)
    return np.nan if vol == 0 else mu / vol


def choose_deadband_delta(
    df,
    train_start=None,
    train_end=None,
    ma_window=50,
    delta_grid=None,
    cost_bps=10,
    lambda_reg=0.001,
):
    """
    Picks a delta from an input grid by maximizing annualized Sharpe while penalizing turnover.
    """

    # Default
    if delta_grid is None:
        delta_grid = [0.0, 0.0025, 0.005, 0.01, 0.02]

    dtrain = df.copy()
    start = pd.to_datetime(train_start)
    end   = pd.to_datetime(train_end)
    dtrain = dtrain.loc[start:end]

    best = {"delta": None, "score": -np.inf, "sharpe": None, "turnover_per_year": None}
    rows = []

    for delta in delta_grid:
        pos_raw = make_trend_positions(dtrain, ma_window=ma_window, delta=delta)
        bt = backtest_from_pos_raw(dtrain, pos_raw, cost_bps=cost_bps)

        s = sharpe_ann(bt["strat_r"])
        turnover_per_year = bt["turnover"].mean() * 252

        # Objective
        score = s - lambda_reg * turnover_per_year

        rows.append((delta, s, turnover_per_year, score))

        if np.isfinite(score) and score > best["score"]:
            best = {
                "delta": float(delta),
                "score": float(score),
                "sharpe": float(s) if np.isfinite(s) else np.nan,
                "turnover_per_year": float(turnover_per_year),
            }

    results = pd.DataFrame(rows, columns=["delta", "train_sharpe", "turnover_per_year", "score"])
    results = results.sort_values("score", ascending=False)

    return best, results

def make_mr_positions(df, z_window=20, a=1.0):
    """
    Mean reversion positions {-1,0,1}. Uses z-score of log price vs rolling mean/std.
    """
    logP = np.log(df["Price"])
    mu = logP.rolling(z_window).mean()
    sd = logP.rolling(z_window).std(ddof=0)
    z = (logP - mu) / sd

    pos = np.select(
        [z < -a, z > a],
        [-1, 1],
        default=0
    )
    return pd.Series(pos, index=df.index, name="pos_mr_raw")

def choose_mr_threshold(
    df,
    train_start=None,
    train_end=None,
    z_window=10,
    a_grid=None,
    cost_bps=10,
    lambda_turnover=0.0001
):
    """
    Similar logic to choose_deadband_delta, but for mean reversion.
    I.e. picks delta from input grid by maximizing annualized Sharpe while penalizing turnover.
    """
    if a_grid is None:
        a_grid = [0.5, 0.75, 1.0, 1.25, 1.5, 2.0]

    dtrain = df.copy()
    start = pd.to_datetime(train_start) if train_start is not None else dtrain.index.min()
    end   = pd.to_datetime(train_end)   if train_end is not None else dtrain.index.max()
    dtrain = dtrain.loc[start:end]

    best = {"a": None, "score": -np.inf, "sharpe": None, "turnover_per_year": None}
    rows = []

    for a in a_grid:
        pos_raw = make_mr_positions(dtrain, z_window=z_window, a=a)
        bt = backtest_from_pos_raw(dtrain, pos_raw, cost_bps=cost_bps)

        s = sharpe_ann(bt["strat_r"])
        turnover_per_year = bt["turnover"].mean() * 252
        score = s - lambda_turnover * turnover_per_year

        rows.append((a, s, turnover_per_year, score))

        if np.isfinite(score) and score > best["score"]:
            best = {
                "a": float(a),
                "score": float(score),
                "sharpe": float(s) if np.isfinite(s) else np.nan,
                "turnover_per_year": float(turnover_per_year)
            }

    results = pd.DataFrame(rows, columns=["a", "train_sharpe", "turnover_per_year", "score"]).sort_values("score", ascending=False)
    return best, results

def metrics(bt, trading_days=252):
    """
    get relevant metrics from a backtest
    """
    r = bt["strat_r"].dropna()
    if len(r) < 2:
        return {}

    total = bt["equity"].iloc[-1] - 1
    ann_ret = (bt["equity"].iloc[-1]) ** (trading_days / len(r)) - 1
    ann_vol = r.std() * np.sqrt(trading_days)
    sharpe = np.nan if ann_vol == 0 else ann_ret / ann_vol

    # Max drawdown
    peak = bt["equity"].cummax()
    dd = bt["equity"] / peak - 1
    max_dd = dd.min()

    avg_turnover = bt["turnover"].mean()
    trades_per_year = avg_turnover * trading_days

    return {
        "Total Return": total,
        "Ann Return": ann_ret,
        "Ann Vol": ann_vol,
        "Sharpe": sharpe,
        "Max Drawdown": max_dd,
        "Avg Daily Turnover": avg_turnover,
        "Turnover/yr (proxy)": trades_per_year
    }


In [25]:
# split
train_start, train_end = "2005-01-04", "2016-12-31"
test_start,  test_end  = "2017-01-01", "2025-12-31"

# calls, note some of these numbers are arbitrary for our purposes, e.g. we just randomly set cost to 10 bps and max turnover to once a day

# signal 1
best_ma, delta_table = choose_deadband_delta(
    df,
    train_start=train_start,
    train_end=train_end,
)
delta_ma = best_ma["delta"] if best_ma["delta"] is not None else 0.005
print("Delta chosen:", best_ma)
# signal 2
best_mr, mr_table = choose_mr_threshold(
    df,
    train_start=train_start,
    train_end=train_end,
)
delta_mr = best_mr["a"] if best_mr["a"] is not None else 1.0
print("MR chosen:", best_mr)


Delta chosen: {'delta': 0.01, 'score': 0.23420924590239867, 'sharpe': 0.2677039688575438, 'turnover_per_year': 33.49472295514512}
MR chosen: {'a': 2.0, 'score': 0.5613274921546707, 'sharpe': 0.5642697085135098, 'turnover_per_year': 29.4221635883905}


It is interesting to note the Sharpe ratio and score is always negative for the mean reversion case UNDER THE REGULAR mean reversion signal (e.g. z < -a = 1; z > a = -1). However, if we invert the signal we find that the Sharpe and scores turn positive; this is intriguing for as it essentially implies that this signal ("the price is noticeably above/below its average") acts as a **regime** signal rather than a mean reversion one.

TLDR: For the case of Brent we see that trends/regimes are more likely to follow when we observe outlier values rather than mean reversion. So truthfully we should just re-label our second function to something more honest, although I guess we wouldn't know this if we didn't first test the mean reversion case.

**Expectations** Let's make this even more concrete. We can calculate the average next-day return for two cases, one when z > a, and one when z < -a (high vs low).

In [26]:
# expectations
z_window = 20
a = 2.0

df = df.copy()
df["logP"] = np.log(df["Price"])
df["r_next"] = df["logP"].diff().shift(-1)   # r_{t+1} aligned to time t

mu = df["logP"].rolling(z_window).mean()
sd = df["logP"].rolling(z_window).std(ddof=0)
df["z"] = (df["logP"] - mu) / sd
high = df["z"] > a
low  = df["z"] < -a
mid  = (df["z"] >= -a) & (df["z"] <= a)

E_high = df.loc[high, "r_next"].mean() * 10000
E_low  = df.loc[low,  "r_next"].mean()* 10000
E_mid  = df.loc[mid,  "r_next"].mean()* 10000

print("Counts:", high.sum(), low.sum(), mid.sum())
print("E[r_{t+1} | z_t > a]   =", E_high)
print("E[r_{t+1} | z_t < -a]  =", E_low)
print("E[r_{t+1} | |z_t|<=a]  =", E_mid)

Counts: 303 343 4657
E[r_{t+1} | z_t > a]   = 17.39217023766222
E[r_{t+1} | z_t < -a]  = -7.681004202530294
E[r_{t+1} | |z_t|<=a]  = 0.2641685170813399


We can see that When z is high, an average of 17.4 bps/day and when low an avergae of -7.7 bps/day. Also worth noting is the "mid" group is positive, indicating a general upward trend. This all supports what we articulated in english, except in a quantitative way 😊

\
Let's get back on track with the actual backtester though.

In [27]:
df_test = df.loc["2017-01-01":"2025-12-31"].copy()

# recompute signals/positions on df_test
df_test["pos_trend_raw"] = make_trend_positions(df_test, ma_window=50, delta=delta_ma)
df_test["pos_mr_raw"]    = make_mr_positions(df_test, z_window=20, a=delta_mr)

bt_trend_test = backtest_from_pos_raw(df_test, df_test["pos_trend_raw"])
bt_mr_test    = backtest_from_pos_raw(df_test, df_test["pos_mr_raw"])

print(metrics(bt_trend_test))
print(metrics(bt_mr_test))

worst = bt_trend_test.nsmallest(10, "strat_r")[["pos", "r", "strat_r"]]
best  = bt_trend_test.nlargest(10, "strat_r")[["pos", "r", "strat_r"]]

print("Worst 10 days:\n", worst)
print("\nBest 10 days:\n", best)

# Also check worst underlying returns days
print("\nLargest abs returns days:\n", df_test.loc[df_test["r"].abs().sort_values(ascending=False).head(10).index, ["Price","r"]])


{'Total Return': np.float64(-0.7431788490604168), 'Ann Return': np.float64(-0.13933446884009892), 'Ann Vol': np.float64(0.46858894676826346), 'Sharpe': np.float64(-0.2973490301063494), 'Max Drawdown': -0.9161334221372163, 'Avg Daily Turnover': np.float64(0.13447218572054315), 'Turnover/yr (proxy)': np.float64(33.88699080157687)}
{'Total Return': np.float64(-0.710225508116252), 'Ann Return': np.float64(-0.12778887848158005), 'Ann Vol': np.float64(0.22714251210657263), 'Sharpe': np.float64(-0.5625934013691941), 'Max Drawdown': -0.7618354101589179, 'Avg Daily Turnover': np.float64(0.12352168199737187), 'Turnover/yr (proxy)': np.float64(31.127463863337713)}
Worst 10 days:
             pos         r   strat_r
Date                               
2020-04-22 -1.0  0.509868 -0.509868
2020-04-02 -1.0  0.352037 -0.352037
2020-05-05 -1.0  0.248039 -0.248039
2020-04-03 -1.0  0.202075 -0.202075
2020-04-29 -1.0  0.144872 -0.144872
2020-04-08 -1.0  0.141176 -0.141176
2022-03-09  1.0 -0.124643 -0.12464

Our strategy is pretty terrible as it is set-up now. We can see average returns of  around -70% each. Upon further analysis, we can see that during a Black Swan event (2020 march-ish) the strategy is exploited and exploited badly, suffering great losses.

My most immediate instinct is to reduce volatility. Let's introduce a function that scales our position down when volatility spikes.

Something like:

$$
\text{pos}_t = \text{pos}_t \cdot \min\left(1,\frac{\sigma_{\text{target}}}{\sigma_t}\right)
$$



In [28]:
# updated function
def backtest_from_pos_raw_v2(df, pos_raw, cost_bps=10, vol_window=10, target_vol_ann=0.10):
    """
    Backtest with volatility targeting (scales position magnitude)

    target_vol_ann: annualized vol target (e.g., 0.20 = 20%/yr)
    """
    out = df[["Price", "r"]].copy()
    out["r"] = out["r"].fillna(0)
    out["pos_raw"] = pos_raw

    # Lag to avoid lookahead
    out["pos"] = out["pos_raw"].shift(1).fillna(0)

    # rolling daily vol
    out["vol_d"] = out["r"].rolling(vol_window).std(ddof=0)

    # target daily vol
    target_vol_d = target_vol_ann / np.sqrt(252)

    # scaling factor
    out["scale"] = (target_vol_d / out["vol_d"]).clip(upper=1.0)
    out["scale"] = out["scale"].replace([np.inf, -np.inf], np.nan).fillna(0)

    # scaled position (continuous)
    out["pos_scaled"] = out["pos"] * out["scale"]

    # Turnover + costs should be based on the position you actually hold
    out["turnover"] = (out["pos_scaled"] - out["pos_scaled"].shift(1)).abs().fillna(0)
    c = cost_bps / 10000.0
    out["cost"] = c * out["turnover"]

    out["strat_r"] = out["pos_scaled"] * out["r"] - out["cost"]
    out["equity"] = (1 + out["strat_r"]).cumprod()
    return out

In [29]:
bt_trend_test = backtest_from_pos_raw_v2(df_test, df_test["pos_trend_raw"])
bt_mr_test    = backtest_from_pos_raw_v2(df_test, df_test["pos_mr_raw"])

# baseline buy and hold
bh_brent = (1 + df_test["r"]).cumprod()
print("Brent buy and hold return:", float(bh_brent.iloc[-1] - 1))

# for export later
df["bh_equity"] = (1 + df_test["r"].fillna(0)).cumprod()

print(metrics(bt_trend_test))
print(metrics(bt_mr_test))

Brent buy and hold return: 0.11626637554585617
{'Total Return': np.float64(0.6661979730065155), 'Ann Return': np.float64(0.057972602421368036), 'Ann Vol': np.float64(0.1011760232894703), 'Sharpe': np.float64(0.5729875570964591), 'Max Drawdown': -0.20683890286106366, 'Avg Daily Turnover': np.float64(0.07450687721350331), 'Turnover/yr (proxy)': np.float64(18.775733057802835)}
{'Total Return': np.float64(-0.07468723347189277), 'Ann Return': np.float64(-0.008531560459549503), 'Ann Vol': np.float64(0.03729059135168325), 'Sharpe': np.float64(-0.22878587199353703), 'Max Drawdown': -0.17543945227047886, 'Avg Daily Turnover': np.float64(0.0495413081017182), 'Turnover/yr (proxy)': np.float64(12.484409641632986)}


With this new function we can see that returns are generally better. However, trend 2 is still typically negative (unless we use the magic number of 5 for it's volatility window). Thus, moving forward we should probably just drop it altogether, i.e. there is no edge to be found unlike with the moving average signal. It's probably redundant anyway since we pivoted from mean reversion.

Let's make a "portfolio" by introducing **WTI**, another crude oil.

In [30]:
# clean
df_wti = pd.read_csv("wti-daily.csv")
df_wti["Date"] = pd.to_datetime(df_wti["Date"])
df_wti = df_wti[df_wti["Date"] >= "2005-01-01"]
df_wti = df_wti.sort_values("Date").drop_duplicates("Date")
df_wti = df_wti[df_wti["Price"].notna() & (df_wti["Price"] > 0)]
df_wti["r"] = df_wti["Price"].pct_change()
df_wti = df_wti.set_index("Date")
df_wti["r"] = df_wti["r"].fillna(0)

# signal 1
# choose delta
best_ma_wti, delta_table_wti = choose_deadband_delta(
    df_wti,
    train_start=train_start,
    train_end=train_end,
)
delta_ma_wti = best_ma_wti["delta"] if best_ma_wti["delta"] is not None else 0.005
#print("wti delta chosen:", best_ma_wti)

# test
df_test_wti = df_wti.loc["2017-01-01":"2025-12-31"].copy()

# positions
df_test_wti["pos_trend_raw"] = make_trend_positions(df_test_wti, ma_window=50, delta=delta_ma_wti)

# backtest
bt_trend_test_wti = backtest_from_pos_raw_v2(df_test_wti, df_test_wti["pos_trend_raw"])
print(metrics(bt_trend_test_wti))

# baseline buy and hold
bh_wti = (1 + df_test_wti["r"]).cumprod()
df_test_wti["bh_equity"] = (1 + df_test_wti["r"].fillna(0)).cumprod()
print("wti buy and hold return:", float(bh_wti.iloc[-1] - 1))

# signal 2, regime continuation (mean reversion inverted)
best_mr_wti, mr_table_wti = choose_mr_threshold(
    df_wti,
    train_start=train_start,
    train_end=train_end,
)
delta_mr_wti = best_mr_wti["a"] if best_mr_wti["a"] is not None else 1.0
df_test_wti["pos_mr_raw"] = make_mr_positions(df_test_wti, z_window=20, a=delta_mr_wti)
bt_rc_test_wti = backtest_from_pos_raw_v2(df_test_wti, df_test_wti["pos_mr_raw"])
print("wti regime continuation signal:", metrics(bt_rc_test_wti))


{'Total Return': np.float64(0.07956886572917243), 'Ann Return': np.float64(0.008619482033134451), 'Ann Vol': np.float64(0.10406900608767787), 'Sharpe': np.float64(0.08282467909679621), 'Max Drawdown': -0.2644602891046628, 'Avg Daily Turnover': np.float64(0.08213113976031902), 'Turnover/yr (proxy)': np.float64(20.697047219600393)}
wti buy and hold return: 0.06530232558140003
wti regime continuation signal: {'Total Return': np.float64(0.20050517917972543), 'Ann Return': np.float64(0.020696628919949944), 'Ann Vol': np.float64(0.08020318896715664), 'Sharpe': np.float64(0.2580524438800713), 'Max Drawdown': -0.2127049285765943, 'Avg Daily Turnover': np.float64(0.10276179456831026), 'Turnover/yr (proxy)': np.float64(25.895972231214188)}


**Firstly** I think its interesting to note that with wti, the "useful" strategy changes. I.e. we can see that the inverted mr function acts as a better signal than the moving average.

Thus I think it would be wise to run seperate strategies depending on stock. So a MA signal for Brent, and a regime continuation signal for WTI.

Now let's combine the two commodities into a portfolio. We'll use even weights as since we're not insider trading (lol) I don't see why we would favor one over the other.

In [31]:
# combine for portfolio
commodities = pd.DataFrame({
    "brent": bt_trend_test["strat_r"],
    "wti": bt_rc_test_wti["strat_r"],
}).dropna()

commodities["port"] = 0.5*commodities["brent"] + 0.5*commodities["wti"]
commodities["port_equity"] = (1 + commodities["port"]).cumprod()
print("Portfolio total return:", commodities["port_equity"].iloc[-1] - 1)
print("corr:", commodities["brent"].corr(commodities["wti"]))

Portfolio total return: 0.35793898125485923
corr: 0.4643803976967573


We can see that the correlation is not that high (46%) and so we can try to pick better weights. Let's first compare metrics for each backtest

In [32]:
# portfolio backtest

# drawdown
roll_max = commodities["port_equity"].cummax()
commodities["port_drawdown"] = commodities["port_equity"] / roll_max - 1

bt_portfolio = pd.DataFrame(index=commodities.index)
bt_portfolio["strat_r"] = commodities["port"]
bt_portfolio["equity"]  = commodities["port_equity"]
# dummy
bt_portfolio["turnover"] = 0.0

print("Brent:", metrics(bt_trend_test))
print("Wti:", metrics(bt_rc_test_wti))
print("Portfolio:", metrics(bt_portfolio))

Brent: {'Total Return': np.float64(0.6661979730065155), 'Ann Return': np.float64(0.057972602421368036), 'Ann Vol': np.float64(0.1011760232894703), 'Sharpe': np.float64(0.5729875570964591), 'Max Drawdown': -0.20683890286106366, 'Avg Daily Turnover': np.float64(0.07450687721350331), 'Turnover/yr (proxy)': np.float64(18.775733057802835)}
Wti: {'Total Return': np.float64(0.20050517917972543), 'Ann Return': np.float64(0.020696628919949944), 'Ann Vol': np.float64(0.08020318896715664), 'Sharpe': np.float64(0.2580524438800713), 'Max Drawdown': -0.2127049285765943, 'Avg Daily Turnover': np.float64(0.10276179456831026), 'Turnover/yr (proxy)': np.float64(25.895972231214188)}
Portfolio: {'Total Return': np.float64(0.35793898125485923), 'Ann Return': np.float64(0.0353903563564808), 'Ann Vol': np.float64(0.07795974212276048), 'Sharpe': np.float64(0.4539568165932725), 'Max Drawdown': -0.21607982651294466, 'Avg Daily Turnover': np.float64(0.0), 'Turnover/yr (proxy)': np.float64(0.0)}


We can see slightly less volatility (Ann Vol) on the WTI stock, so if our goal is to **minimize risk by increasing diversification** we can use an inverse volatility weighting system.

\
**Something like**:


$$
\tilde{w}_B = \frac{1}{\sigma_B}, \qquad \tilde{w}_W = \frac{1}{\sigma_W}
$$

Where
$$
w_B = \frac{\tilde{w}_B}{\tilde{w}_B + \tilde{w}_W}
$$

$$
w_W = 1 - w_B
$$




In [33]:
sigB = commodities["brent"].std()
sigW = commodities["wti"].std()

wB = (1/sigB) / ((1/sigB) + (1/sigW))
wW = 1 - wB

print("weights:", wB, wW)

commodities["port"] = wB*commodities["brent"] + wW*commodities["wti"]
commodities["port_equity"] = (1 + commodities["port"]).cumprod()

weights: 0.4419195342543593 0.5580804657456406


Probably not optimized for performance (e.g. returns), but our goal *was* diversification so let's keep the 0.44-0.56 split as its atleast not arbitrary.

Okay, Let's make some visualizations in Tableau. We'll need to create a table to export with relevant details.

In [34]:
export = pd.DataFrame(index=commodities.index)
# prices
export["brent_price"] = df_test["Price"]
export["wti_price"]   = df_test_wti["Price"]

# backtest returns
export["brent_strat_r"] = bt_trend_test["strat_r"]
export["wti_strat_r"]   = bt_rc_test_wti["strat_r"]
export["port_strat_r"]  = commodities["port"]

# equity curves
export["brent_equity"] = bt_trend_test["equity"]
export["wti_equity"]   = bt_rc_test_wti["equity"]
export["port_equity"]  = commodities["port_equity"]

# positions
export["brent_pos"] = bt_trend_test.get("pos_scaled", bt_trend_test.get("pos"))
export["wti_pos"]   = bt_rc_test_wti.get("pos_scaled", bt_rc_test_wti.get("pos"))

export["brent_scale"] = bt_trend_test.get("scale")
export["wti_scale"]   = bt_rc_test_wti.get("scale")

# clean + save
export = export.reset_index().rename(columns={"index": "Date"})
export.to_csv("commodities_tableau.csv", index=False)